In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import subprocess
import numpy as np
import itertools
from itertools import count

In [ ]:
class Policy(nn.Module):
    def __init__(self, state_space, action_space):
        super(Policy, self).__init__()
        self.state_space = state_space 
        self.action_space = action_space
        self.l1 = nn.Sequential(nn.Linear(self.state_space, 6*self.state_space), nn.ReLU())
        self.l2 = nn.Sequential(nn.Linear(6*self.state_space, 10*self.action_space), nn.ReLU())
        self.l3 = nn.Linear(10*self.action_space, self.action_space)
        
    def forward(self, x):    
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [ ]:
class ERenv():
    import platform
    import numpy as np
    def __init__(self, exe, init_m = 100, nrange=(100,200), mc=10):
        """
            nrange is half-open [a,b)
        """
        self.exe    = exe
        self.mc     = mc
        self.nrange = (100,200)
        self.init_m = init_m
        self.state  = [np.random.randint(*self.nrange), init_m]
        # self.pyver  = platform.python_version()[:3] == 3.6
        
    def step(self, p):
        if True: 
            tmp = subprocess.run(['./main', str(self.state[0]), str(10) ,str(p)],stdout=subprocess.PIPE) ## 3.6
        else:
            tmp = subprocess.run(['./main', str(self.state[0]), str(10) ,str(p)],capture_output=True) ## 3.7
        res = float(tmp.stdout.split()[0])
        self.state[0] = np.random.randint(*self.nrange)
        self.state[1] -= 1
        return self.state.copy(), float(tmp.stdout.split()[0]), self.state[1]==0
    
    def reset(self):
        self.state[0] = np.random.randint(*self.nrange)
        self.state[1] = self.init_m
        return self.state.copy()
    
    def run(self, state, p):
        tmp = subprocess.run(['./main', str(state[0]), str(10) ,str(p)],stdout=subprocess.PIPE) ## 3.6
        return float(tmp.stdout.split()[0])

In [ ]:
def simu(env, policy_net, batch_size=8, gamma=0.99):
    state_pool, action_pool, reward_pool = [], [], []
    for b in range(batch_size):
        done  = False
        state = env.reset()
        cur_reward_pool = []
        for t in itertools.count():
            if done: break
            state_pool.append(state)
            state  = torch.tensor(state).float()
            action = policy_net(state)
            action_pool.append(action)
            state, reward, done, *_ = env.step(nn.Sigmoid()(action).item())
            cur_reward_pool.append(reward)
        
        # discount reward
        running_mean = 0
        for i in reversed(range(len(cur_reward_pool))):
            running_mean = running_mean*gamma + cur_reward_pool[i]
            cur_reward_pool[i] = running_mean
        reward_pool.extend(cur_reward_pool)
    
    # normalize reward
    avg, std = np.mean(reward_pool), np.std(reward_pool)
    reward_pool = list(map(lambda x: (x-avg)/std, reward_pool))
    
    return state_pool, action_pool, reward_pool


In [ ]:
def update_policy(policy_net, optim, state_pool, action_pool, reward_pool):
    optim.zero_grad()
    for s, a, r in zip(state_pool, action_pool, reward_pool):
        m    = torch.distributions.Normal(nn.Sigmoid()(a), torch.tensor([0.0001]))
        loss = -m.log_prob(a)*r
        loss.backward()
    optim.step()

In [ ]:
env = ERenv('./main', init_m=10)
policy_net = Policy(2, 1)
optim = torch.optim.SGD(policy_net.parameters(), lr=0.001)
for i in range(100):
    print(i)
    tmp_act = policy_net(torch.tensor([155, 5]).float()).sigmoid().item()
    print( tmp_act, env.run([155,5], tmp_act) )
    update_policy(policy_net, optim, *simu(env, policy_net, 4))